## Reinforcement Learning - **2048**

### 2048

- Tabla 4x4
- Puteri ale lui 2
- Mutari posibile: sus, jos, stanga, dreapta {0, 1, 2, 3}
- Dupa fiecare mutare se adauga un tile nou: 90% probabilitate 2, 10% probabilitate 4
- Cand facem merge intre doua tiles cu aceeasi valoare, obtinem un tile cu valoarea 2*valoarea initiala
- Jocul se termina cand nu mai exista mutari posibile sau obtinem valoarea 2048

### Reinforcement Learning
Reinforcement Learning - antrenarea unor modele (ML) pentru a lua o serie de decizii. Agentul invata sa indeplineasca un obiectiv intr-un mediu complex si nesigur. AI-ul primeste recompense sau penalizari pentru actiunile pe care le indeplineste, scopul fiind sa maximizeze recompensa totala.


### Codul

In [0]:
import random as rd
import numpy as np
from keras.models import Model, load_model
from keras.layers import Input, Dense, LeakyReLU, Dropout
from keras.optimizers import Adam
import math
from copy import deepcopy

### Functii pentru joc

In [0]:
# initializeaza tabla
def new_game(n):
    return np.zeros([n, n])


# adauga 2 sau 4 pe tabla
def add_two(state):
    empty_cells = []

    for i in range(len(state)):
        for j in range(len(state[0])):
            if state[i][j] == 0:
                empty_cells.append((i, j))

    if len(empty_cells) == 0:
        return state

    index_pair = empty_cells[rd.randint(0, len(empty_cells) - 1)]

    prob = rd.random()
    if prob >= 0.9:
        state[index_pair[0]][index_pair[1]] = 4
    else:
        state[index_pair[0]][index_pair[1]] = 2
    return state


# verifica daca mai exista mutari: tile-uri goale, mutari la stanga sau dreapta, mutari in sus sau jos
def game_over(state):
    for i in range(len(state) - 1):
        for j in range(len(state[0]) - 1):
            if state[i][j] == state[i + 1][j] or state[i][j + 1] == state[i][j]:
                return False

    for i in range(len(state)):
        for j in range(len(state[0])):
            if state[i][j] == 0:
                return False

    for k in range(len(state) - 1):
        if state[len(state) - 1][k] == state[len(state) - 1][k + 1]:
            return False

    for j in range(len(state) - 1):
        if state[j][len(state) - 1] == state[j + 1][len(state) - 1]:
            return False

    return True


def reverse(state):
    new_state = []
    for i in range(len(state)):
        new_state.append([])
        for j in range(len(state[0])):
            new_state[i].append(state[i][len(state[0]) - j - 1])
    return new_state


def transpose(state):
    return np.transpose(state)


def cover_up(state):
    new = [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]
    done = False
    for i in range(4):
        count = 0
        for j in range(4):
            if state[i][j] != 0:
                new[i][count] = state[i][j]
                if j != count:
                    done = True
                count += 1
    return new, done


def merge(state):
    done = False
    score = 0
    for i in range(4):
        for j in range(3):
            if state[i][j] == state[i][j + 1] and state[i][j] != 0:
                state[i][j] *= 2
                score += state[i][j]
                state[i][j + 1] = 0
                done = True
    return state, done, score


def up(game):
    game = transpose(game)
    game, done = cover_up(game)
    temp = merge(game)
    game = temp[0]
    done = done or temp[1]
    game = cover_up(game)[0]
    game = transpose(game)
    return game, done, temp[2]


def down(game):
    game = reverse(transpose(game))
    game, done = cover_up(game)
    temp = merge(game)
    game = temp[0]
    done = done or temp[1]
    game = cover_up(game)[0]
    game = transpose(reverse(game))
    return game, done, temp[2]


def left(game):
    game, done = cover_up(game)
    temp = merge(game)
    game = temp[0]
    done = done or temp[1]
    game = cover_up(game)[0]
    return game, done, temp[2]


# right move
def right(game):
    game = reverse(game)
    game, done = cover_up(game)
    temp = merge(game)
    game = temp[0]
    done = done or temp[1]
    game = cover_up(game)[0]
    game = reverse(game)
    return game, done, temp[2]

### Mutari posibile

In [0]:
controls = {0: up, 1: left, 2: right, 3: down}

### Functii ajutatoare

In [0]:
# schimba matricea in alta matrice cu puterile corespondente ale lui 2
def change_values(state):
    power_mat = np.zeros(shape=(1, 4, 4, 16), dtype=np.float32)
    for i in range(4):
        for j in range(4):
            if state[i][j] == 0:
                power_mat[0][i][j][0] = 1.0
            else:
                power = int(math.log(state[i][j], 2))
                power_mat[0][i][j][power] = 1.0
    return power_mat


# numara tile-urile goale
def count_empty_cells(state):
    count = 0
    for i in range(len(state)):
        for j in range(len(state)):
            if state[i][j] == 0:
                count += 1
    return count

### Agentul cu parametrii si modelul


Lista de parametrii:
- state_size - 16
- action_size - 4
- Gamma - 0.9
- Epsilon - 0.9
- Learning Rate - 0.001

Modelul:
- Layer 1: Input, primeste dimensiunea de intrare
- Layer 2: Dense cu 64 units si functia de activare 'linear'
- Layer 3: LeakyReLU cu alpha 0.001
- Layer 4: Dropout cu rate 0.125
- Layer 5: Dense cu 32 units
- Layer 6: LeakyReLU cu alpha 0.001
- Layer 7: Dense cu 4 units (dimensiunea output-ului) si functia de activare 'linear'
- Optimizer: Adam cu learning rate = 0.001
- Loss: Mean Squared Error (mse)

In [0]:
class DQNAgent:
    def __init__(self):
        self.state_size = 16
        self.action_size = 4
        self.learning_rate = 0.001
        self.model = None
        self.gamma = 0.9
        self.epsilon = 0.9

    def build_model(self):
        input_dimensions = (self.action_size, self.action_size, self.state_size)
        output_dimensions = self.action_size

        inputs = Input(shape=input_dimensions, name='inputs')
        hidden_1 = Dense(64, activation='linear', name='hidden_1')(inputs)
        hidden_2 = LeakyReLU(0.001, name="hidden_2")(hidden_1)
        hidden_3 = Dropout(rate = 0.125, name="hidden_3")(hidden_2)
        hidden_4 = Dense(32, name='hidden_4')(hidden_3)
        hidden_5 = LeakyReLU(0.001, name="hidden_5")(hidden_4)
        outputs = Dense(output_dimensions, activation='linear', name='outputs')(hidden_5)

        self.model = Model(inputs=inputs, outputs=outputs)
        optimizer = Adam(learning_rate=self.learning_rate)
        self.model.compile(loss='mse', optimizer=optimizer, metrics=['acc'])

### Antrenarea
Pentru fiecare episod se realizeaza urmatorii pasi:
1. Se initializeaza tabla si se adauga doua valori in aceasta (2 sau 4 - 90% probabilitate 2, 10% probabilitate 4).
2. Cat timp jocul nu este terminat:
- Modelul este folosit pentru prezicerea celor 4 valori din stare curenta
- Vectorul de valori este sortat descrescator si este creat un alt vector cu indecsii, care reprezinta valorile actiunilor
- Un numar random intre 0 si 1 este generat si se compara cu epsilon (parametrul agentului):
  - Daca numarul generat este mai mic decat epsilon, atunci actiunea este random:
    - O actiune random este aleasa din cele posibile si se aplica pe starea curenta, obtinandu-se noua stare
    - Este calculata recompensa 
    - Modelul este folosit pentru prezicerea celor 4 valori din noua stare
    - Este luata valoarea maxima din vectorul de valori obtinut
    - Label-ul pentru actiune este calculat: recompensa + gamma (parametrul agentului) * valoarea maxima calculata in pasul anterior
  - Daca numarul generat este mai mare decat epsilon, atunci actiunea nu este random:
    - Este aleasa actiunea cu valoarea cea mai mare din vectorul creat anterior si se aplica pe starea curenta, obtinandu-se noua stare
    - Este calculata recompensa
    - Modelul este folosit pentru prezicerea celor 4 valori din noua stare
    - Este luata valoarea maxima din vectorul de valori obtinut
    - Label-ul pentru actiune este calculat: recompensa + gamma (parametrul agentului) * valoarea maxima calculata in pasul anterior
- Valoarea lui epsilon este modificata: daca epsilon este mai mare decat 0.1, epsilon ia valoarea epsilon / 1.005


In [0]:
def train(n_episodes, agent):
    maximum = -1
    episode = -1
    best_board = new_game(4)
    total_iters = 1

    for ep in range(n_episodes+1):
        global board
        board = new_game(4)
        add_two(board)
        add_two(board)
        finish = False
        total_score = 0
        while not finish:
            prev_board = deepcopy(board)
            state = deepcopy(board)
            state = change_values(state)
            state = np.array(state, dtype=np.float32).reshape(1, 4, 4, 16)
            control_scores = agent.model.predict(state)
            control_buttons = np.flip(np.argsort(control_scores), axis=1)
            control_buttons = control_buttons[0][0]
            control_scores = control_scores[0][0]
            labels = deepcopy(control_scores[0])
            num = rd.uniform(0, 1)
            prev_max = np.max(prev_board)
            if num < agent.epsilon:
                legal_moves = list()
                for i in range(4):
                    temp_board = deepcopy(prev_board)
                    temp_board, _, _ = controls[i](temp_board)
                    if np.array_equal(temp_board, prev_board):
                        continue
                    else:
                        legal_moves.append(i)
                if len(legal_moves) == 0:
                    finish = True
                    continue
                con = rd.sample(legal_moves, 1)[0]
                temp_state = deepcopy(prev_board)
                temp_state, _, score = controls[con](temp_state)
                total_score += score
                finish = game_over(temp_state)
                empty1 = count_empty_cells(prev_board)
                empty2 = count_empty_cells(temp_state)
                if not finish:
                    temp_state = add_two(temp_state)
                board = deepcopy(temp_state)
                next_max = np.max(temp_state)
                labels[con] = (math.log(next_max, 2)) * 0.1
                if next_max == prev_max:
                    labels[con] = 0
                labels[con] += (empty2 - empty1)
                temp_state = change_values(temp_state)
                temp_state = np.array(temp_state, dtype=np.float32).reshape(1, 4, 4, 16)
                temp_scores = agent.model.predict(temp_state)
                max_qvalue = np.max(temp_scores)
                labels[con] = (labels[con] + agent.gamma * max_qvalue)
            else:
                for con in control_buttons[0]:
                    prev_state = deepcopy(prev_board)
                    temp_state, _, score = controls[con](prev_state)
                    if np.array_equal(prev_board, temp_state):
                        labels[con] = 0
                        continue
                    empty1 = count_empty_cells(prev_board)
                    empty2 = count_empty_cells(temp_state)
                    temp_state = add_two(temp_state)
                    board = deepcopy(temp_state)
                    total_score += score
                    next_max = np.max(temp_state)
                    labels[con] = (math.log(next_max, 2)) * 0.1
                    if next_max == prev_max:
                        labels[con] = 0
                    labels[con] += (empty2 - empty1)
                    temp_state = change_values(temp_state)
                    temp_state = np.array(temp_state, dtype=np.float32).reshape(1, 4, 4, 16)
                    temp_scores = agent.model.predict(temp_state)
                    max_qvalue = np.max(temp_scores)
                    labels[con] = (labels[con] + agent.gamma * max_qvalue)
                    break
                if np.array_equal(prev_board, board):
                    finish = True

            if (ep > 10000) or (agent.epsilon > 0.1 and total_iters % 2500 == 0):
                agent.epsilon = agent.epsilon / 1.005

            total_iters += 1

        print("Episode " + str(ep) + " finished with score " + str(total_score) + ", board : " + str(board) +
              ", epsilon  : " + str(agent.epsilon))

        if maximum < total_score:
            maximum = total_score
            episode = ep
            best_board = board

    print("Maximum Score: " + str(maximum) + " ,Episode: " + str(episode) + " ,Board: " + str(best_board))
    agent.model.save("Model.h5")

### Testarea/Jucarea unui joc
- Daca se da un path catre un model deja existent, se aplica load_model si apoi se joaca jocul.
- Daca nu se da un path, se face antrenarea pentru numarul de episoade dat ca parametru si apoi se joaca jocul.

In [0]:
def test(n_episodes=5000, model_path=None):
    agent = DQNAgent()
    if model_path:
        print(f"Load model from: {model_path}")
        agent.model = load_model(model_path)
    else:
        agent.build_model()
        train(n_episodes, agent)
    print("Start playing...")
    board = new_game(4)
    board = add_two(board)
    board = add_two(board)
    finish = False
    total_score = 0
    while not finish:
        prev_board = deepcopy(board)
        state = deepcopy(board)
        state = change_values(state)
        state = np.array(state, dtype=np.float32).reshape(1, 4, 4, 16)
        control_scores = agent.model.predict(state)
        control_buttons = np.flip(np.argsort(control_scores), axis=1)
        control_buttons = control_buttons[0][0]
        control_scores = control_scores[0][0]
        labels = deepcopy(control_scores[0])
        num = rd.uniform(0, 1)
        prev_max = np.max(prev_board)
        if num < agent.epsilon:
            legal_moves = list()
            for i in range(4):
                temp_board = deepcopy(prev_board)
                temp_board, _, _ = controls[i](temp_board)
                if np.array_equal(temp_board, prev_board):
                    continue
                else:
                    legal_moves.append(i)
            if len(legal_moves) == 0:
                finish = True
                continue
            con = rd.sample(legal_moves, 1)[0]
            temp_state = deepcopy(prev_board)
            temp_state, _, score = controls[con](temp_state)
            total_score += score
            finish = game_over(temp_state)
            empty1 = count_empty_cells(prev_board)
            empty2 = count_empty_cells(temp_state)
            if not finish:
                temp_state = add_two(temp_state)
            board = deepcopy(temp_state)
            next_max = np.max(temp_state)
            labels[con] = (math.log(next_max, 2)) * 0.1
            if next_max == prev_max:
                labels[con] = 0
            labels[con] += (empty2 - empty1)
            temp_state = change_values(temp_state)
            temp_state = np.array(temp_state, dtype=np.float32).reshape(1, 4, 4, 16)
            temp_scores = agent.model.predict(temp_state)
            max_qvalue = np.max(temp_scores)
            labels[con] = (labels[con] + agent.gamma * max_qvalue)
        else:
            for con in control_buttons[0]:
                prev_state = deepcopy(prev_board)
                temp_state, _, score = controls[con](prev_state)
                if np.array_equal(prev_board, temp_state):
                    labels[con] = 0
                    continue
                empty1 = count_empty_cells(prev_board)
                empty2 = count_empty_cells(temp_state)
                temp_state = add_two(temp_state)
                board = deepcopy(temp_state)
                total_score += score
                next_max = np.max(temp_state)
                labels[con] = (math.log(next_max, 2)) * 0.1
                if next_max == prev_max:
                    labels[con] = 0
                labels[con] += (empty2 - empty1)
                temp_state = change_values(temp_state)
                temp_state = np.array(temp_state, dtype=np.float32).reshape(1, 4, 4, 16)
                temp_scores = agent.model.predict(temp_state)
                max_qvalue = np.max(temp_scores)
                labels[con] = (labels[con] + agent.gamma * max_qvalue)
                break
            if np.array_equal(prev_board, board):
                finish = True

        if agent.epsilon > 0.1:
            agent.epsilon = agent.epsilon / 1.005

    print("Finished with score " + str(total_score) + ", board: " + str(board))

In [9]:
test(20000)

Streaming output truncated to the last 5000 lines.
 [64.  8.  4.  2.]
 [32.  4.  2.  4.]], epsilon  : 4.94e-322
Episode 18076 finished with score 1576.0, board : [[ 16.   2.  32.   2.]
 [128.  32.  16.   8.]
 [ 16.  64.   4.   2.]
 [ 32.   8.   2.   4.]], epsilon  : 4.94e-322
Episode 18077 finished with score 1456.0, board : [[32.0, 8.0, 4.0, 2], [128.0, 16.0, 8.0, 4.0], [2.0, 64.0, 16.0, 8.0], [8.0, 2.0, 32.0, 2.0]], epsilon  : 4.94e-322
Episode 18078 finished with score 824.0, board : [[16.  8.  2.  4.]
 [64. 16.  4.  2.]
 [16. 64. 16.  4.]
 [ 4.  2.  8.  2.]], epsilon  : 4.94e-322
Episode 18079 finished with score 1448.0, board : [[32.0, 64.0, 8.0, 4.0], [128.0, 16.0, 4.0, 2.0], [32.0, 2.0, 8.0, 4.0], [16.0, 8.0, 4.0, 2]], epsilon  : 4.94e-322
Episode 18080 finished with score 644.0, board : [[16.0, 4.0, 2.0, 4.0], [64.0, 16.0, 8.0, 2.0], [16.0, 4.0, 32.0, 4], [4.0, 16.0, 8.0, 2.0]], epsilon  : 4.94e-322
Episode 18081 finished with score 1736.0, board : [[ 64.   8.   2.  16.]
 [128.

### Rezultate
Antrenarea a fost facuta pentru 20.000 de episoade.
Cel mai mare scor obtinut cu modelul actual a fost 8900 la episodul 13.102.
Tile-ul maxim a fost 512 in cele mai multe cazuri, o singura data a fost 1024.

A fost incercat si ca valoarea lui epsilon sa ramana constanta si s-a obtinut doar un scor de 4972 la episodul 18.619.

Cu schimbarea functiei de activare:
- 'linear' pe toate layerele: scor maxim 5976 la episodul 17.165
- 'relu' pe toate layerele: scor maxim 6208 la episodul 10.555

Cu schimbarea optimizer-ului:
- RMSprop: scor maxim 5156 la episodul 13.553

Prima incercare de model a fost cu primul layer Dense cu 128 de units in loc de 64, scorul maxim obtinut fiind 5922 la episodul 17.615.

Dupa obtinerea modelului actual, s-a incercat imbunatatirea prin adaugarea unor layere Conv2D:
- layer Conv2D pe 64 units si kernel (3, 3) si Conv2D pe 32 units si kernel (2, 2) + modelul actual: scor maxim 3468 la episodul 49
- layer Conv2D pe 128 units si kernel (3, 3) si Conv2D pe 64 units si kernel (2, 2) + modelul actual fara layerele LeakyReLU si Dropout: scorul maxim 3412 la episodul 12.004
- layer Conv2D pe 64 units si kernel (3, 3) si Conv2D pe 32 units si kernel (2, 2) + modelul actual fara layerele LeakyReLU si Dropout: prima data a fost obtinut scorul maxim 9284 la episodul 10.130, dar la o a doua incercare scorul maxim a fost doar 4720 la episodul 17.590

  Pentru ca dupa prima incercare scorul maxim obtinut a fost destul de mare, s-a incercat imbunatatirea:
  - cu functia de activare 'linear' pe toate layerele: scor maxim 4476 la episodul 328
  - cu functia de activare 'relu' pe toate layerele: scor maxim 5576 la episodul 2359
  - cu optimizer RMSprop: scor maxim 4880 la episodul 13.243




### Concluzie
Limitatii:
Din cauza dimensiunea tablii (4x4): 
- cand tabla se umple, devine foarte greu sa se aleaga mutarea corecta. Modelul poate astepta noi tile-uri, dar tabla se umple si jocul este pierdut, chiar daca a invatat
- foarte multe stari disponibile

